# 📘 代理架构 10：心智循环

欢迎来到我们系列的第十个notebook。今天，我们探索**心智循环**架构，这是一种受认知科学启发的模式，模拟人类思维的迭代、自我反思性质。

与简单的刺激-响应代理不同，心智循环代理维护一个内部"心智状态"，它在每次交互中更新。这允许代理在多个回合中推理、反思其过去的行动，并随着时间推移调整其策略。

### 定义
一个**Simulator**或**Mental-Model-in-the-Loop** 架构 涉及代理 使用n internal 模型其environment来simulate outcomes的potential actions before executing any的them. 这允许 代理来perform what-if 分析, anticipate consequences,和refine它的plan为safety和有效的ness.

### 高级工作流程

1. **Observe:** 代理 observes current state的real environment.
2. **Propose Action:** 基于它的goals和current state, 代理's planning module generates 一个high-level proposed action或策略.
3. **Simulate:** 代理 forks current state的environment into 一个sandboxed simulation. It applies proposed action和runs simulation forward来observe 一个range的possible outcomes.
4. **Assess & Refine:** 代理 analyzes results从simulation. Did action lead来desired outcome? Were there unforeseen negative consequences? 基于 这个assessment,它refines它的initial proposal into 一个最终的, concrete action.
5. **Execute:** 代理 executes final, refined action在*real* environment.
6. **Repeat:** loop begins again从new state的real environment.

### 何时使用/应用场景
* **Robotics:** Simulating 一个grasp或一个path before moving 一个physical arm来avoid collisions或damage.
* **高风险 Decision-Making:** 在finance, simulating 一个trade's impact在一个portfolio under different market conditions. 在healthcare, simulating 一个treatment plan's potential effects.
* **Complex Game AI:** 一个AI在一个策略 game simulating several moves ahead来choose optimal one.

### 优点和缺点
* **优点：**
 * **Safety & Risk Reduction:** Massively reduces chance的harmful或costly mistakes通过vetting actions在安全的 environment first.
 * **Improved 性能:** Leads来more robust和well-considered decisions通过allowing为lookahead和planning.
* **缺点：**
 * **Simulation-Reality Gap:** 有效的ness 是entirely dependent在fidelity的simulator. 如果 模型的world 是inaccurate, 代理's计划may 是based在false assumptions.
 * **Computational Cost:** Running simulations, especially multiple scenarios, 是computationally expensive和slower than acting directly.

## 阶段0：基础与设置

我们将安装 libraries和设置我们的environment.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv numpy

In [2]:
import os
import random
import numpy as np from typing import List, Dict, Any, Optional from dotenv import load_dotenv

# Pydanticfordata modeling from pydantic import BaseModel, Field

# LangChain components from langchain_nebius import ChatNebius from langchain_core.prompts import ChatPromptTemplate

# LangGraph components from langgraph import StateGraph, END from typing_extensions import TypedDict

# 用于美观打印 from rich.console import Console from rich.markdown import Markdown from rich.table import Table

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Simulator (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"] for var in required_vars:
 if var not in os.environ:
 print(f"警告: Environment variable {var} not set.")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：构建模拟器环境

首先，我们需要创建"world"我们的代理 将interact with. 我们将构建一个`MarketSimulator` class 那个manages state的stock, 一个portfolio,和includes 一个`step` function来advance time. 这将serve 作为both我们的"real world"和sandbox用于我们的代理's simulations.

In [ ]:
console = Console()

class Portfolio(BaseModel):
 cash: float = 10000.0
 shares: int = 0
 
 def value(self, current_price: float) -> float:
 return self.cash + self.shares * current_price

class MarketSimulator(BaseModel):
 """A 简单的 simulation的stock marketforone asset."""
 day: int = 0
 price: float = 100.0
 volatility: float = 0.1 # Standard deviationforprice changes
 drift: float = 0.01 # General trend
 market_news: str = "Market is stable."
 portfolio: Portfolio = Field(default_factory=Portfolio)

 def step(self, action: str, amount: float = 0.0):
 """Advance simulationpassone day, executing 一个trade first."""
 # 1. Execute trade
 if action == "buy": # amount is number的shares
 shares_to_buy = int(amount)
 cost = shares_to_buy * self.price
 if self.portfolio.cash >= cost:
 self.portfolio.shares += shares_to_buy
 self.portfolio.cash -= cost
 elif action == "sell": # amount is number的shares
 shares_to_sell = int(amount)
 if self.portfolio.shares >= shares_to_sell:
 self.portfolio.shares -= shares_to_sell
 self.portfolio.cash += shares_to_sell * self.price
 
 # 2. Update market price (Geometric Brownian Motion)
 daily_return = np.random.normal(self.drift, self.volatility)
 self.price *= (1 + daily_return)
 
 # 3. Advance time
 self.day += 1
 
 # 4. Potentially update news
 if random.random() < 0.1: # 10% chance的new news
 self.market_news = random.choice(["Positive earnings报告expected.", "New competitor enters market.", "Macroeconomic outlook is strong.", "Regulatory concerns are growing."])
 # News affects drift
 if "Positive"inself.market_newsor"strong"inself.market_news:
 self.drift = 0.05
 else:
 self.drift = -0.05
 else:
 self.drift = 0.01 # Revert来normal drift

 def get_state_string(self) -> str:
 return f"Day {self.day}: Price=${self.price:.2f}, News: {self.market_news}\nPortfolio: ${self.portfolio.value(self.price):.2f} ({self.portfolio.shares} shares, ${self.portfolio.cash:.2f} cash)"

print("Market simulator environment defined successfully.")

Market simulator environment 定义成功.


## 阶段2： 构建 Simulator Agent

Now 我们将 使用Lang图来orchestrate `Observe -> Propose -> Simulate -> Refine -> Execute` 工作流程。 我们将定义Pydantic模型为LLM's outputs来ensure structured communication between steps.

In [4]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.4)

# Pydanticmodelforstructured LLM outputs
class ProposedAction(BaseModel):
 """high-level策略proposedpassanalyst.""" 
 策略: str = Field(description="A high-level trading 策略, e.g., 'buy aggressively', 'sell cautiously', 'hold'.")
 reasoning: str = Field(description="Brief reasoningforproposed 策略.")

class FinalDecision(BaseModel):
 """final, concrete action来be executed."""
 action: str = Field(description="final action来take: 'buy', 'sell',or'hold'.")
 amount: float = Field(description="number的shares来buyorsell. 应该be 0 if holding.")
 reasoning: str = Field(description="最终 reasoning, referencing simulation results.")

# LangGraph State
class AgentState(TypedDict):
 real_market: MarketSimulator
 proposed_action: Optional[ProposedAction]
 simulation_results: Optional[List[Dict]]
 final_decision: Optional[最终Decision]

# Graph Nodes
def propose_action_node(state: AgentState) -> Dict[str, Any]:
 """Observes marketandproposes 一个high-level 策略."""
 console.print("--- 🧐 分析师提出行动 ---")
 prompt = ChatPromptTemplate.from_template(
 "你is一位敏锐的 财务分析师. 基于 current market state, propose 一个trading 策略.\n\nMarket State:\n{market_state}"
 )
 proposer_llm = llm.with_structured_output(ProposedAction)
 chain = prompt | proposer_llm
 proposal = chain.invoke({"market_state": state['real_market'].get_state_string()})
 console.print(f"[yellow]Proposal:[/yellow] {proposal.策略}. [italic]Reason: {proposal.reasoning}[/italic]")
 return{"proposed_action": proposal}

def run_simulation_node(state: AgentState) -> Dict[str, Any]:
 """Runs proposed 策略in一个sandboxed simulation."""
 console.print("--- 🤖 Running Simulations ---")
 策略= state['proposed_action'].策略
 num_simulations = 5
 simulation_horizon = 10 # days
 result= []

 foriinrange(num_simulations):
 # IMPORTANT: Create 一个deep copy来not affect real market state
 simulated_market = state['real_market'].model_copy(deep=True)
 initial_value = simulated_market.portfolio.value(simulated_market.price)

 # Translate 策略一个concrete actionforsimulation
 if "buy"in策略:
 action = "buy"
 # Aggressively = 25%的cash, Cautiously = 10%
 amount = (simulated_market.portfolio.cash * (0.25 if "aggressively"in策略 else 0.1)) / simulated_market.price
 elif "sell"in策略:
 action = "sell"
 # Aggressively = 25%的shares, Cautiously = 10%
 amount = simulated_market.portfolio.shares * (0.25 if "aggressively"in策略 else 0.1)
 else:
 action = "hold"
 amount = 0
 
 # Run simulation forward
 simulated_market.step(action, amount)
 for_inrange(simulation_horizon - 1):
 simulated_market.step("hold") # Just hold in...之后initial action
 
 final_value = simulated_market.portfolio.value(simulated_market.price)
 results.append({"sim_num": i+1, "initial_value": initial_value, "final_value": final_value, "return_pct": (final_value - initial_value) / initial_value * 100})
 
 console.print("[cyan]Simulation 完成. results 将be passed来risk manager.[/cyan]")
 return{"simulation_results": results}

def refine_and_decide_node(state: AgentState) -> Dict[str, Any]:
 """分析 simulation resultsandmakes 一个最终的, refined decision."""
 console.print("--- 🧠 Risk Manager Refining Decision ---")
 results_summary = "\n".join([f"Sim {r['sim_num']}: Initial=${r['initial_value']:.2f}, 最终=${r['final_value']:.2f}, Return={r['return_pct']:.2f}%"forrinstate['simulation_results']])
 
 prompt = ChatPromptTemplate.from_template(
 "你is一个cautious risk manager. Your analyst proposed 一个策略. You have运行simulations来test it. 基于 potential outcomes,制作一个最终的, concrete decision. Ifresultare highly variableornegative, reduce risk (e.g., buy/sell fewer shares,orhold).\n\nInitial Proposal: {proposal}\n\nSimulation results:\n{results}\n\nReal Market State:\n{market_state}"
 )
 decider_llm = llm.with_structured_output(最终Decision)
 chain = prompt | decider_llm
 final_decision = chain.invoke({
 "proposal": state['proposed_action'].策略,
 "results": results_summary,
 "market_state": state['real_market'].get_state_string()
 })
 console.print(f"[green]最终 Decision:[/green] {final_decision.action} {final_decision.amount:.0f} shares. [italic]Reason: {final_decision.reasoning}[/italic]")
 return{"final_decision": final_decision}

def execute_in_real_world_node(state: AgentState) -> Dict[str, Any]:
 """Executes final decisioninreal market environment."""
 console.print("--- 🚀 ExecutinginReal World ---")
 decision = state['final_decision']
 real_market = state['real_market']
 real_market.step(decision.action, decision.amount)
 console.print(f"[bold]Execution 完成. New market state:[/bold]\n{real_market.get_state_string()}")
 return{"real_market": real_market}

# Build graph
workflow = StateGraph(AgentState)
workflow.add_node("propose", propose_action_node)
workflow.add_node("simulate", run_simulation_node)
workflow.add_node("refine", refine_and_decide_node)
workflow.add_node("execute", execute_in_real_world_node)

workflow.set_entry_point("propose")
workflow.add_edge("propose", "simulate")
workflow.add_edge("simulate", "refine")
workflow.add_edge("refine", "execute")
workflow.add_edge("execute", END)

simulator_agent = workflow.compile()
print("Simulator-in-the-loop 代理 Graph编译成功.")

Simulator-in-the-loop 代理 图编译成功.


## 阶段3： 演示nstration

让我们在市场上运行我们的代理几天. 我们将 start与some好的news和see how它reacts, then introduce some坏的news.

In [5]:
real_market = MarketSimulator()
console.print("--- Initial Market State ---")
console.print(real_market.get_state_string())

# --- Day 1 Run ---
console.print("\n--- Day 1: Good News Hits! ---")
real_market.market_news = "Positive earnings报告expected."
real_market.drift = 0.05
initial_state = {"real_market": real_market}
final_state = simulator_agent.invoke(initial_state)
real_market = final_state['real_market']

# --- Day 2 Run ---
console.print("\n--- Day 2: Bad News Hits! ---")
real_market.market_news = "New competitor enters market."
real_market.drift = -0.05
initial_state = {"real_market": real_market}
final_state = simulator_agent.invoke(initial_state)
real_market = final_state['real_market']

--- Initial Market State ---


Day 0: Price=$100.00, News: Market是stable.
Portfolio: $10000.00 (0 shares, $10000.00 cash)



--- Day 1: Good News Hits! ---


--- 🧐 分析师提出行动 ---
Proposal: buy aggressively. Reason: positive earnings报告is 一个strong bullish signal,andmarket是already stable. 这is一个good opportunity来enter 一个position before price potentially rises further.
--- 🤖 Running Simulations ---
Simulation 完成. 结果s 将be passed来risk manager.
--- 🧠 Risk Manager 改进 Decision ---
最终 Decision: buy 20 shares. Reason: simulations confirm 一个strong upward trend,withall场景resultingin一个positive return. analyst's proposal来buy aggressively是validated.我will执行一个significant但not excessive purchase的20 shares来capitalizeinexpected price increase while maintaining 一个cash reserve.
--- 🚀 ExecutinginReal World ---
Execution 完成. New market state:
Day 1: Price=$99.16, News: Market是stable.
Portfolio: $7983.18 (20 shares, $8000.00 cash)



--- Day 2: Bad News Hits! ---


--- 🧐 分析师提出行动 ---
Proposal: sell cautiously. Reason: entry的new competitor introduces significant uncertaintyandpotential downside risk. 虽然price hasn't dropped dramatically yet, it's prudent来reduce exposure.
--- 🤖 Running Simulations ---
Simulation 完成. 结果s 将be passed来risk manager.
--- 🧠 Risk Manager 改进 Decision ---
最终 Decision: sell 5 shares. Reason: simulations显示一个high degree的varianceand一个negative average return, confirming analyst's concerns. initial proposal来sell cautiously是sound.我将de-risk portfoliopassselling 5 shares (25%的position)来lockinsome cashandreduce exposure来potential downsidefromnew competitor.
--- 🚀 ExecutinginReal World ---
Execution 完成. New market state:
Day 2: Price=$93.81, News: Market是stable.
Portfolio: $9802.90 (15 shares, $8395.73 cash)


### 结果分析

agent's behavior 展示 value的simulator loop:

- **在Day 1 (Good News):**
 - *Analyst* proposed 一个aggressive buy, seeing opportunity.
 - *Simulator* confirmed high probability的positive outcome.
 - *Risk Manager* translated aggressive策略into 一个concrete, substantial purchase (20 shares)，但didn't risk entire cash balance.

- **在Day 2 (Bad News):**
 - *Analyst* correctly identified新的risk和proposed 一个cautious sell.
 - *Simulator* likely showed 一个mix的outcomes,与some场景showing 一个sharp drop和others 一个recovery, confirming uncertainty.
 - *Risk Manager*, seeing variance和negative average return在simulations, made 一个prudent decision来reduce position (selling 5 shares) rather than panic-selling entire holding. 这是 一个much more nuanced action than 一个simple rule-based 代理 可能take.

一个天真的代理 without simulation loop 可能有bought too much在day 1和then sold everything在day 2, incurring higher transaction costs和potentially missing 一个recovery. Our simulator 代理 acted more like 一个real-world trader, making 一个probabilistic bet和then hedging 那个bet when new信息changed risk pro文件。

## 总结

在这个notebook中，我们have built 一个强大的 代理 架构 使用n internal **simulator**来test和refine它的actions before committing them. 通过creating 一个loop的`Propose -> Simulate -> Refine -> Execute`，我们enabled我们的代理来perform sophisticated risk 分析和make more nuanced, safer decisions在一个dynamic environment.

这种模式 是一个cornerstone的building 代理那个can operate safely和有效的ly在real world.。..的能力执行what-if 分析在一个internal "mental 模型" 允许 代理来anticipate consequences, avoid costly errors,和develop more robust strategies. 虽然fidelity的simulator 是一个关键的 dependency (the "simulation-reality gap"), 这个architecture 提供 一个clear和extensible framework为building responsible, action-taking AI.